In [1]:
import pandas as pd
import numpy as np

In [3]:
fraud_df = pd.read_csv("../data/Fraud_Data.csv")
ip_df = pd.read_csv("../data/IpAddress_to_Country.csv")

In [4]:
fraud_df.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [5]:
ip_df.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [6]:
fraud_df['ip_int'] = fraud_df['ip_address'].astype('int64')

In [7]:
# Ensure IP range boundaries are integers
# ------------------------------------------------------------
# These columns define the start and end of each IP range.
# They MUST be integers to allow numeric comparison.

ip_df['lower_bound_ip_address'] = ip_df['lower_bound_ip_address'].astype('int64')
ip_df['upper_bound_ip_address'] = ip_df['upper_bound_ip_address'].astype('int64')

In [8]:
# Function to map a single IP integer to a country

def get_country(ip_int):
    """
    Given an integer IP address, find the country
    whose IP range contains this IP.
    """

    # Filter IP ranges where:
    # lower_bound <= ip_int <= upper_bound

    row = ip_df[
        (ip_df['lower_bound_ip_address'] <= ip_int) &
        (ip_df['upper_bound_ip_address'] >= ip_int)
    ]
    # If a matching range is found, return its country
    if not row.empty:

        return row.iloc[0]['country']
    # If no match is found, label as 'Unknown'
    return 'Unknown'

# Apply the IP-to-country mapping for every transaction
# ------------------------------------------------------------
# Each row in fraud_df gets a country based on its IP address
fraud_df['country'] = fraud_df['ip_int'].apply(get_country)

In [9]:
# Count total transactions per country
transaction_counts = fraud_df['country'].value_counts()
transaction_counts.head(20)

country
United States                      58049
Unknown                            21966
China                              12038
Japan                               7306
United Kingdom                      4490
Korea Republic of                   4162
Germany                             3646
France                              3161
Canada                              2975
Brazil                              2961
Italy                               1944
Australia                           1844
Netherlands                         1680
Russian Federation                  1616
India                               1310
Taiwan; Republic of China (ROC)     1237
Mexico                              1121
Sweden                              1090
Spain                               1027
South Africa                         838
Name: count, dtype: int64

In [10]:
# Analyze fraud rate by country
# ------------------------------------------------------------
# 'class' = 1 → fraudulent transaction
# Mean gives fraud rate per country

# Compute fraud rate per country
fraud_rate = fraud_df.groupby('country')['class'].mean().sort_values(ascending=False)
fraud_rate.head(10)

country
Turkmenistan             1.000000
Namibia                  0.434783
Sri Lanka                0.419355
Luxembourg               0.388889
Virgin Islands (U.S.)    0.333333
Ecuador                  0.264151
Tunisia                  0.262712
Peru                     0.260504
Bolivia                  0.245283
Kuwait                   0.233333
Name: class, dtype: float64

In [11]:
# Merge counts and fraud rate into a single DataFrame
country_summary = pd.DataFrame({
    'transactions': transaction_counts,
    'fraud_rate': fraud_rate
}).sort_values(by='fraud_rate', ascending=False)

country_summary.head(20)

,transactions,fraud_rate
country,,
Turkmenistan,1,1.000000
Namibia,23,0.434783
Sri Lanka,31,0.419355
Luxembourg,72,0.388889
Virgin Islands (U.S.),3,0.333333
Ecuador,106,0.264151
Tunisia,118,0.262712
Peru,119,0.260504
Bolivia,53,0.245283
